# Trajectory Generation Visualization

Interactive 3D visualization of corridor-constrained trajectories generated by `generate_trajectories.py`.

Trajectories navigate through the corridor network, turning at intersections.

In [1]:
import json
import subprocess
import tempfile
import os
import xml.etree.ElementTree as ET
import plotly.graph_objects as go
from typing import List, Dict, Tuple


def generate_corridors(grid_size=400, num_ew=2, num_ns=2, width="20",
                       spacing="100", altitude_min=0, altitude_max=400, seed=None):
    """Run generate_corridors.py and return parsed corridors."""
    cmd = [
        "python3", "generate_corridors.py",
        "--grid-size", str(grid_size),
        "--num-ew", str(num_ew),
        "--num-ns", str(num_ns),
        "--width", str(width),
        "--spacing", str(spacing),
        "--altitude-min", str(altitude_min),
        "--altitude-max", str(altitude_max),
    ]
    if seed is not None:
        cmd.extend(["--seed", str(seed)])

    result = subprocess.run(cmd, capture_output=True, text=True)
    corridors = [json.loads(line) for line in result.stdout.strip().split('\n') if line]
    return corridors


def generate_buildings(corridors: List[Dict], grid_size=400, num_buildings=20,
                       width_x="30-50", width_y="30-50", height="50-150", seed=None):
    """Run generate_buildings.py and return parsed buildings."""
    with tempfile.NamedTemporaryFile(mode='w', suffix='.ndjson', delete=False) as f:
        for c in corridors:
            json.dump(c, f)
            f.write('\n')
        corridors_file = f.name

    try:
        cmd = [
            "python3", "generate_buildings.py",
            "-c", corridors_file,
            "-n", str(num_buildings),
            "--grid-size", str(grid_size),
            "--width-x", str(width_x),
            "--width-y", str(width_y),
            "--height", str(height),
        ]
        if seed is not None:
            cmd.extend(["--seed", str(seed)])

        result = subprocess.run(cmd, capture_output=True, text=True)
        buildings = [json.loads(line) for line in result.stdout.strip().split('\n') if line]
        return buildings
    finally:
        os.unlink(corridors_file)


def generate_trajectories(corridors: List[Dict], grid_size=400, hosts=5,
                          min_duration=500, speed="5-15", altitude="30-100",
                          waypoint_interval="30-60", seed=None):
    """Run generate_trajectories.py and return parsed trajectories."""
    with tempfile.NamedTemporaryFile(mode='w', suffix='.ndjson', delete=False) as f:
        for c in corridors:
            json.dump(c, f)
            f.write('\n')
        corridors_file = f.name

    with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as f:
        output_file = f.name

    try:
        cmd = [
            "python3", "generate_trajectories.py",
            "-c", corridors_file,
            "--hosts", str(hosts),
            "--grid-size", str(grid_size),
            "--min-duration", str(min_duration),
            "--speed", str(speed),
            "--altitude", str(altitude),
            "--waypoint-interval", str(waypoint_interval),
            "-o", output_file,
        ]
        if seed is not None:
            cmd.extend(["--seed", str(seed)])

        result = subprocess.run(cmd, capture_output=True, text=True)

        # Parse XML output
        trajectories = parse_trajectory_xml(output_file)
        return trajectories
    finally:
        os.unlink(corridors_file)
        if os.path.exists(output_file):
            os.unlink(output_file)


def parse_trajectory_xml(filepath: str) -> Dict[int, Dict]:
    """Parse TurtleMobility XML and return trajectories."""
    tree = ET.parse(filepath)
    root = tree.getroot()

    trajectories = {}
    for movement in root.findall('movement'):
        traj_id = int(movement.get('id'))
        waypoints = []
        speed = 10.0

        for elem in movement:
            if elem.tag == 'set':
                x = float(elem.get('x'))
                y = float(elem.get('y'))
                z = float(elem.get('z'))
                speed = float(elem.get('speed', 10.0))
                waypoints.append((x, y, z))
            elif elem.tag == 'moveto':
                x = float(elem.get('x'))
                y = float(elem.get('y'))
                z = float(elem.get('z'))
                waypoints.append((x, y, z))

        trajectories[traj_id] = {
            'waypoints': waypoints,
            'speed': speed
        }

    return trajectories


def create_corridor_mesh(corridor: Dict, grid_size: float, color: str, opacity: float = 0.15):
    """Create a 3D mesh for a corridor."""
    c = corridor["center"]
    w = corridor["width"] / 2
    z_min = corridor["altitude_min"]
    z_max = corridor["altitude_max"]

    if corridor["direction"] == "EW":
        x = [0, grid_size, grid_size, 0, 0, grid_size, grid_size, 0]
        y = [c-w, c-w, c+w, c+w, c-w, c-w, c+w, c+w]
    else:
        x = [c-w, c+w, c+w, c-w, c-w, c+w, c+w, c-w]
        y = [0, 0, grid_size, grid_size, 0, 0, grid_size, grid_size]

    z = [z_min, z_min, z_min, z_min, z_max, z_max, z_max, z_max]
    i = [0, 0, 4, 4, 0, 1, 0, 3, 1, 2, 4, 5]
    j = [1, 2, 5, 6, 1, 5, 3, 7, 2, 6, 5, 6]
    k = [2, 3, 6, 7, 4, 4, 4, 4, 5, 5, 1, 2]

    return go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color=color,
        opacity=opacity,
        name=f"{corridor['direction']} corridor",
        hoverinfo="skip",
        showlegend=False
    )


def create_building_mesh(building: Dict, color: str = "gray", opacity: float = 0.6):
    """Create a 3D mesh for a building."""
    cx, cy = building["x"], building["y"]
    wx, wy = building["width_x"] / 2, building["width_y"] / 2
    h = building["height"]

    x = [cx-wx, cx+wx, cx+wx, cx-wx, cx-wx, cx+wx, cx+wx, cx-wx]
    y = [cy-wy, cy-wy, cy+wy, cy+wy, cy-wy, cy-wy, cy+wy, cy+wy]
    z = [0, 0, 0, 0, h, h, h, h]

    i = [0, 0, 4, 4, 0, 1, 0, 3, 1, 2, 4, 5]
    j = [1, 2, 5, 6, 1, 5, 3, 7, 2, 6, 5, 6]
    k = [2, 3, 6, 7, 4, 4, 4, 4, 5, 5, 1, 2]

    return go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color=color,
        opacity=opacity,
        hoverinfo="skip",
        showlegend=False
    )


# Color palette for trajectories
TRAJECTORY_COLORS = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd",
    "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"
]


def visualize_trajectories(corridors: List[Dict], trajectories: Dict[int, Dict],
                           buildings: List[Dict] = None, grid_size: float = 400,
                           title: str = "Corridor-Constrained Trajectories",
                           show_corridors: bool = True, show_buildings: bool = True,
                           show_waypoints: bool = True):
    """Create interactive 3D visualization of trajectories."""
    fig = go.Figure()

    # Add ground plane
    fig.add_trace(go.Mesh3d(
        x=[0, grid_size, grid_size, 0],
        y=[0, 0, grid_size, grid_size],
        z=[0, 0, 0, 0],
        i=[0, 0], j=[1, 2], k=[2, 3],
        color="lightgreen",
        opacity=0.3,
        name="Ground",
        hoverinfo="skip",
        showlegend=False
    ))

    # Add buildings
    if show_buildings and buildings:
        for building in buildings:
            gray = 120 + (building['id'] % 4) * 15
            color = f"rgb({gray}, {gray}, {gray})"
            fig.add_trace(create_building_mesh(building, color=color))

    # Add corridors (semi-transparent)
    if show_corridors:
        for corridor in corridors:
            color = "rgba(100, 150, 255, 0.1)" if corridor["direction"] == "EW" else "rgba(255, 100, 100, 0.1)"
            fig.add_trace(create_corridor_mesh(corridor, grid_size, color, opacity=0.1))

    # Add trajectories
    for traj_id, traj_data in trajectories.items():
        waypoints = traj_data['waypoints']
        speed = traj_data['speed']
        color = TRAJECTORY_COLORS[traj_id % len(TRAJECTORY_COLORS)]

        xs = [w[0] for w in waypoints]
        ys = [w[1] for w in waypoints]
        zs = [w[2] for w in waypoints]

        # Add trajectory line
        fig.add_trace(go.Scatter3d(
            x=xs, y=ys, z=zs,
            mode='lines',
            line=dict(color=color, width=3),
            name=f"Host {traj_id} ({speed:.1f} m/s)",
            hovertemplate=f"Host {traj_id}<br>x: %{{x:.1f}}<br>y: %{{y:.1f}}<br>z: %{{z:.1f}}<extra></extra>"
        ))

        # Add waypoint markers
        if show_waypoints:
            fig.add_trace(go.Scatter3d(
                x=xs, y=ys, z=zs,
                mode='markers',
                marker=dict(color=color, size=2, opacity=0.5),
                showlegend=False,
                hoverinfo="skip"
            ))

        # Add start marker
        fig.add_trace(go.Scatter3d(
            x=[xs[0]], y=[ys[0]], z=[zs[0]],
            mode='markers',
            marker=dict(color=color, size=8, symbol='diamond'),
            name=f"Host {traj_id} start",
            showlegend=False,
            hovertemplate=f"Host {traj_id} START<br>x: %{{x:.1f}}<br>y: %{{y:.1f}}<br>z: %{{z:.1f}}<extra></extra>"
        ))

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="X (East) [m]",
            yaxis_title="Y (North) [m]",
            zaxis_title="Altitude [m]",
            aspectmode="data",
            camera=dict(eye=dict(x=1.5, y=1.5, z=0.8))
        ),
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
        margin=dict(l=0, r=0, t=40, b=0),
        height=700
    )

    return fig

## Example 1: Basic Trajectories

5 hosts navigating a 2x2 corridor grid.

In [2]:
corridors = generate_corridors(num_ew=2, num_ns=2, width="20", spacing="120", seed=42)
trajectories = generate_trajectories(corridors, hosts=5, min_duration=300, seed=42)

print(f"Generated {len(trajectories)} trajectories")
for tid, tdata in trajectories.items():
    print(f"  Host {tid}: {len(tdata['waypoints'])} waypoints, {tdata['speed']:.1f} m/s")

fig = visualize_trajectories(corridors, trajectories, title="Basic: 5 Hosts, 2x2 Corridors")
fig.show()

Generated 5 trajectories
  Host 0: 75 waypoints, 11.4 m/s
  Host 1: 73 waypoints, 10.1 m/s
  Host 2: 35 waypoints, 5.7 m/s
  Host 3: 93 waypoints, 14.8 m/s
  Host 4: 71 waypoints, 10.5 m/s


## Example 2: Dense Urban Environment

Trajectories with buildings visible.

In [3]:
corridors = generate_corridors(num_ew=3, num_ns=3, width="15", spacing="100", seed=1)
buildings = generate_buildings(corridors, num_buildings=25, width_x="20-35", width_y="20-35", height="60-150", seed=1)
trajectories = generate_trajectories(corridors, hosts=5, min_duration=300, altitude="40-80", seed=1)

print(f"Corridors: {len(corridors)}, Buildings: {len(buildings)}, Trajectories: {len(trajectories)}")

fig = visualize_trajectories(corridors, trajectories, buildings=buildings,
                              title="Dense Urban: 5 Hosts with Buildings")
fig.show()

Corridors: 6, Buildings: 25, Trajectories: 5


## Example 3: Long Duration Trajectories

Trajectories lasting at least 600 seconds.

In [4]:
corridors = generate_corridors(num_ew=2, num_ns=2, width="25", spacing="130", seed=10)
trajectories = generate_trajectories(corridors, hosts=3, min_duration=600, speed="8-12", seed=10)

print(f"Generated {len(trajectories)} long trajectories")
for tid, tdata in trajectories.items():
    wps = tdata['waypoints']
    # Calculate total distance
    dist = sum(((wps[i+1][0]-wps[i][0])**2 + (wps[i+1][1]-wps[i][1])**2)**0.5 
               for i in range(len(wps)-1))
    duration = dist / tdata['speed']
    print(f"  Host {tid}: {len(wps)} waypoints, {tdata['speed']:.1f} m/s, ~{duration:.0f}s")

fig = visualize_trajectories(corridors, trajectories, title="Long Duration: ~600s Trajectories")
fig.show()

Generated 3 long trajectories
  Host 0: 129 waypoints, 10.3 m/s, ~609s
  Host 1: 125 waypoints, 9.3 m/s, ~604s
  Host 2: 142 waypoints, 10.9 m/s, ~604s


## Example 4: High Waypoint Density

More waypoints with smaller intervals for smoother paths.

In [5]:
corridors = generate_corridors(num_ew=2, num_ns=2, width="20", spacing="120", seed=5)
trajectories = generate_trajectories(
    corridors, hosts=3, min_duration=200,
    waypoint_interval="10-20",  # Dense waypoints
    seed=5
)

print(f"High density waypoints:")
for tid, tdata in trajectories.items():
    print(f"  Host {tid}: {len(tdata['waypoints'])} waypoints")

fig = visualize_trajectories(corridors, trajectories, 
                              title="High Density Waypoints (10-20m intervals)",
                              show_waypoints=True)
fig.show()

High density waypoints:
  Host 0: 156 waypoints
  Host 1: 182 waypoints
  Host 2: 100 waypoints


## Example 5: Low Waypoint Density

Fewer waypoints with larger intervals.

In [6]:
corridors = generate_corridors(num_ew=2, num_ns=2, width="20", spacing="120", seed=5)
trajectories = generate_trajectories(
    corridors, hosts=3, min_duration=200,
    waypoint_interval="60-100",  # Sparse waypoints
    seed=5
)

print(f"Low density waypoints:")
for tid, tdata in trajectories.items():
    print(f"  Host {tid}: {len(tdata['waypoints'])} waypoints")

fig = visualize_trajectories(corridors, trajectories,
                              title="Low Density Waypoints (60-100m intervals)",
                              show_waypoints=True)
fig.show()

Low density waypoints:
  Host 0: 27 waypoints
  Host 1: 14 waypoints
  Host 2: 18 waypoints


## Example 6: Different Seeds, Same Parameters

Shows variation in trajectory patterns.

In [7]:
corridors = generate_corridors(num_ew=2, num_ns=2, width="20", spacing="120", seed=42)

for seed in [1, 42, 99]:
    trajectories = generate_trajectories(corridors, hosts=3, min_duration=200, seed=seed)
    fig = visualize_trajectories(corridors, trajectories, 
                                  title=f"Trajectory Seed={seed}",
                                  show_waypoints=False)
    fig.show()

## Example 7: Variable Altitude

Wider altitude range shows more 3D variation.

In [8]:
corridors = generate_corridors(num_ew=2, num_ns=2, width="20", spacing="120", seed=7)
trajectories = generate_trajectories(
    corridors, hosts=4, min_duration=250,
    altitude="20-150",  # Wide altitude range
    seed=7
)

fig = visualize_trajectories(corridors, trajectories,
                              title="Wide Altitude Range (20-150m)")
fig.show()

## Example 8: Speed Variation

Different speed ranges affect trajectory length for same duration.

In [9]:
corridors = generate_corridors(num_ew=2, num_ns=2, width="20", spacing="120", seed=8)

# Slow drones
traj_slow = generate_trajectories(corridors, hosts=2, min_duration=200, speed="3-5", seed=8)
# Fast drones  
traj_fast = generate_trajectories(corridors, hosts=2, min_duration=200, speed="12-18", seed=8)

print("Slow drones (3-5 m/s):")
for tid, tdata in traj_slow.items():
    print(f"  Host {tid}: {len(tdata['waypoints'])} waypoints, {tdata['speed']:.1f} m/s")

print("\nFast drones (12-18 m/s):")
for tid, tdata in traj_fast.items():
    print(f"  Host {tid}: {len(tdata['waypoints'])} waypoints, {tdata['speed']:.1f} m/s")

# Combine for visualization
combined = {}
for tid, tdata in traj_slow.items():
    combined[tid] = tdata
for tid, tdata in traj_fast.items():
    combined[tid + 2] = tdata

fig = visualize_trajectories(corridors, combined,
                              title="Speed Comparison: Hosts 0-1 slow, 2-3 fast")
fig.show()

Slow drones (3-5 m/s):
  Host 0: 26 waypoints, 3.5 m/s
  Host 1: 16 waypoints, 4.0 m/s

Fast drones (12-18 m/s):
  Host 0: 61 waypoints, 13.4 m/s
  Host 1: 55 waypoints, 12.2 m/s


## Example 9: Full Urban Scene

Complete visualization with corridors, buildings, and trajectories.

In [10]:
# Create urban environment
corridors = generate_corridors(num_ew=3, num_ns=3, width="15", spacing="100", seed=100)
buildings = generate_buildings(corridors, num_buildings=30, 
                                width_x="25-40", width_y="25-40", 
                                height="50-180", seed=100)
trajectories = generate_trajectories(corridors, hosts=6, min_duration=400,
                                      altitude="50-120", seed=100)

print(f"Urban scene: {len(corridors)} corridors, {len(buildings)} buildings, {len(trajectories)} drones")

fig = visualize_trajectories(corridors, trajectories, buildings=buildings,
                              title="Full Urban Scene: 6 Drones in City",
                              show_waypoints=False)
fig.show()

Urban scene: 6 corridors, 30 buildings, 6 drones
